In [2]:
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached optree-0.16.0-cp312-cp312-win_amd64.whl.metadata (31 kB)
  Using cached tensorboard

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
import random
import string

def is_palindrome(s):
    return s == s[::-1]

def generate_palindrome_data(n_samples=1000, max_len=10):
    data = []
    labels = []
    alphabet = list(string.ascii_lowercase[:10]) 
    for _ in range(n_samples):
        length = random.randint(3, max_len)
        if random.random() > 0.5:
            half = [random.choice(alphabet) for _ in range(length // 2)]
            if length % 2 == 0:
                word = half + half[::-1]
            else:
                word = half + [random.choice(alphabet)] + half[::-1]
            label = 1
        else:
            word = [random.choice(alphabet) for _ in range(length)]
            label = int(is_palindrome(word))
        data.append(word)
        labels.append(label)
    return data, labels


In [5]:
char_to_int = {ch: i+1 for i, ch in enumerate(string.ascii_lowercase[:10])}

def encode_data(data):
    return [[char_to_int[ch] for ch in word] for word in data]

raw_data, labels = generate_palindrome_data()
encoded_data = encode_data(raw_data)
padded_data = pad_sequences(encoded_data, maxlen=10, padding='post') 
X = np.array(padded_data)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [10]:
print("Ejemplo de entrada:", X_train[0])
print("Etiqueta:", y_train[0])
print("Número de clases:", np.unique(y_train, return_counts=True))

Ejemplo de entrada: [9 7 9 0 0 0 0 0 0 0]
Etiqueta: 1
Número de clases: (array([0, 1]), array([390, 410], dtype=int64))


In [13]:
from tensorflow.keras import Input
from tensorflow.keras.models import Model

inputs = Input(shape=(10,))
x = Embedding(input_dim=11, output_dim=8)(inputs)
x = SimpleRNN(16)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 10, 8)          │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 16)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 505 (1.97 KB)

 Trainable params: 505 (1.97 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4975 - loss: 0.6967 - val_accuracy: 0.5000 - val_loss: 0.6972
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5379 - loss: 0.6895 - val_accuracy: 0.4750 - val_loss: 0.6986
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5092 - loss: 0.6902 - val_accuracy: 0.5375 - val_loss: 0.6945
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5280 - loss: 0.6846 - val_accuracy: 0.5000 - val_loss: 0.6954
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5668 - loss: 0.6796 - val_accuracy: 0.5000 - val_loss: 0.6943
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5841 - loss: 0.6671 - val_accuracy: 0.4750 - val_loss: 0.7013
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5852 - loss: 0.6675 - val_accuracy: 0.4750 - val_loss: 0.7057
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5996 - loss: 0.6584 - val_accuracy: 0.4750 - val_loss

In [15]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy en test: {acc:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5065 - loss: 0.7328  
Accuracy en test: 0.49


In [16]:
def predict_palindrome(word):
    encoded = [char_to_int.get(ch, 0) for ch in word]
    padded = pad_sequences([encoded], maxlen=10, padding='post')
    prediction = model.predict(padded)[0][0]
    print(f"Palabra: {''.join(word)} - Probabilidad de palíndromo: {prediction:.2f}")

predict_palindrome(list("abcba"))
predict_palindrome(list("abcdef"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Palabra: abcba - Probabilidad de palíndromo: 0.68
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Palabra: abcdef - Probabilidad de palíndromo: 0.45
